In [39]:
def format_variable(name, array):
    lines = f"const {name}= [\n"
    lines += ",\n".join([f"\t{element}" for element in array])
    lines += "\n]\n\n"
    return lines

## Auxiliar code to generate inverse normal CDF interpolation nodes 

In [45]:
import numpy as np
from scipy.stats import norm

MESH_SIZE = 10000
N_STD = 8.0

# Compute mesh and the probability values
quantiles = np.linspace(-N_STD, N_STD, MESH_SIZE)
probabilities = norm.cdf(quantiles)

# Save quantiles and probabilities as a JS file
with open("./src/functions/util/normal-cdf-data.js", "w") as file:
    lines = format_variable("X", probabilities)
    lines += format_variable("Y", quantiles)
    lines += "module.exports = { X, Y };"
    file.write(lines)

### Computation of k, for a given threshold to approximate poisson distribution with normal distribution

In [73]:
import math
from scipy.stats import poisson, norm

epsilon = 1e-16  # Umbral para KL

def poisson_pmf(lambda_, k):
    """Probabilidad de Poisson."""
    fact = math.factorial(int(k))
    return np.power(lambda_,k) * math.exp(-lambda_) / fact

def normal_pdf(mean, std, k):
    """Densidad de la distribución Normal."""
    return (1.0 / (std * math.sqrt(2.0 * math.pi))) * math.exp(-(np.square(k - mean)) / (2 * np.square(std)))

def kl_divergence_poisson_normal(lambda_):
    """Calcula la divergencia KL entre Poisson y Normal para un rango de k."""
    mean = lambda_
    std = math.sqrt(lambda_)
    kl = 0.0
    k = 0.0

    while True:
        pk = poisson_pmf(lambda_, k)
        qk = normal_pdf(mean, std, k)
        k += 1.0

        if pk > epsilon and qk > epsilon:  # Evitar log(0) o divisiones por cero
            kl += pk * math.log(pk / qk)
        else:
            break
    
    return kl

def find_lambda_for_epsilon(tol):
    """Encuentra el valor de lambda para el cual KL(P||Q) <= epsilon."""
    low, high = 1.0, 1000.0  # Rango inicial de búsqueda
    best_lambda = None
    
    while high - low > tol:
        mid = (low + high) / 2.0
        kl = kl_divergence_poisson_normal(mid)
        print(low, mid, high, high-low, kl)
        
        if kl <= epsilon:
            best_lambda = mid
            high = mid  # Seguir buscando un lambda menor que cumpla
        else:
            low = mid  # Aumentar el rango de búsqueda
    
    return best_lambda

lambda_approx = find_lambda_for_epsilon(0.1)
print(f"El valor de lambda para KL <= {epsilon} es: {lambda_approx:.4f}")


1.0 500.5 1000.0 999.0 0.0
1.0 250.75 500.5 499.5 0.0
1.0 125.875 250.75 249.75 0.0
1.0 63.4375 125.875 124.875 0.0
1.0 32.21875 63.4375 62.4375 0.002628318833316491
32.21875 47.828125 63.4375 31.21875 0.0
32.21875 40.0234375 47.828125 15.609375 0.0
32.21875 36.12109375 40.0234375 7.8046875 0.0023401817738232036
36.12109375 38.072265625 40.0234375 3.90234375 0.0
36.12109375 37.0966796875 38.072265625 1.951171875 0.0
36.12109375 36.60888671875 37.0966796875 0.9755859375 0.0023085481587149124
36.60888671875 36.852783203125 37.0966796875 0.48779296875 0.0
36.60888671875 36.7308349609375 36.852783203125 0.243896484375 0.002300772976225045
36.7308349609375 36.79180908203125 36.852783203125 0.1219482421875 0.0022969049986084683
El valor de lambda para KL <= 1e-16 es: 36.8528


### Computation of k, for a given threshold to approximate chi2 distribution with normal distribution

In [72]:
import numpy as np
from scipy.stats import chi2, norm
import matplotlib.pyplot as plt
from scipy.special import rel_entr

def kl_divergence_chi2_normal(k, num_points=1000000):
    chi2_dist = chi2(k)
    normal_dist = norm(loc=k, scale=np.sqrt(2.0 * k))
    
    x = np.linspace(chi2.ppf(epsilon, k), chi2.ppf(1.0-epsilon, k), num_points)
    
    p = chi2_dist.pdf(x)
    q = normal_dist.pdf(x)
    
    return rel_entr(p, q).sum()

def find_k_for_epsilon(tol=epsilon):
    low, high = 1, 10000000
    best_k = None
    
    while high - low > tol:
        mid = (low + high) / 2
        kl = kl_divergence_chi2_normal(mid)
        print(low, high, best_k, kl)
        
        if kl <= epsilon:
            best_k = mid
            high = mid 
        else:
            low = mid 
    
    return best_k

epsilon=1e-2
k_approx = find_k_for_epsilon(epsilon)
print(k_approx)
print(f"El valor de k para KL <= {epsilon} es: {k_approx:.6f}")


1 10000000 None 7.353439123321153e-06
1 5000000.5 5000000.5 2.0306253601679385e-05
1 2500000.75 2500000.75 5.664171853134581e-05
1 1250000.875 1250000.875 0.0001600924718357389
1 625000.9375 625000.9375 0.00045303955197460217
1 312500.96875 312500.96875 0.0012815423591038882
1 156250.984375 156250.984375 0.0036246967752530035
1 78125.9921875 78125.9921875 0.010252150061093935
39063.49609375 78125.9921875 78125.9921875 0.005580597290055379
39063.49609375 58594.744140625 58594.744140625 0.007335867170344989
39063.49609375 48829.1201171875 48829.1201171875 0.008591857330120137
39063.49609375 43946.30810546875 43946.30810546875 0.009361000938967044
39063.49609375 41504.902099609375 41504.902099609375 0.009789690281648203
39063.49609375 40284.19909667969 40284.19909667969 0.010016485223876742
39673.847595214844 40284.19909667969 40284.19909667969 0.009901998984766716
39673.847595214844 39979.023345947266 39979.023345947266 0.009958969680877094
39673.847595214844 39826.435470581055 39826.435